In [1]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [2]:
STORM_NUMBER = 14
WINTER = "2024-2025"
TITLE = "17 February 2025"
SUBTITLE = "7 AM 18 February 2025"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2025, 2, 17, 15)
ets = datetime(2025, 2, 18, 19)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,WY,CYS,66.8,-106.350000,41.380000,POINT (-529418.416 -382020.879),True,0,True,LSR,-26,5
1,WY,CYS,51.1,-106.380000,41.330000,POINT (-532329.871 -387372.714),False,1,False,LSR,-26,5
2,WY,CYS,42.9,-106.230000,41.360000,POINT (-519600.362 -385004.234),True,2,True,LSR,-25,5
3,WY,CYS,34.4,-106.230000,41.240000,POINT (-520575.681 -398314.232),True,3,True,LSR,-26,4
4,WY,CYS,31.9,-106.910000,41.010000,POINT (-579306.646 -419277.617),True,4,True,LSR,-27,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2151,IA,DVN,0.0,-91.490770,41.653370,POINT (705700.389 -335494.907),True,IA-JH-54,True,COCORAHS,14,6
2152,IA,DVN,0.0,-91.529572,41.678623,POINT (702216.226 -333031.222),True,IA-JH-58,True,COCORAHS,14,6
2153,IA,DVN,0.0,-91.571810,41.651110,POINT (699033.899 -336438.358),True,IA-JH-61,True,COCORAHS,14,6
2154,IA,DVN,0.0,-91.270037,42.105643,POINT (718731.11 -283500.909),True,IA-JN-16,True,COCORAHS,15,8


In [12]:
def main():
    setpoints = [

    ]
    #if setpoints:
    #    add_setpoints(setpoints)
    cull = [
        'LSXI4',
        
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
        ('CLDI4', 3),
        ('OSEI4', 0.1),
        (248, 1),
        ('IA-LC-12', 0.1),
        ('MO-ND-6', 3),
        ('NCYN1', 3),
        ('OTWI4', 0.0001),
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0., upper=10)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250218.png")
    mp.close()


main()

     state  wfo  val        lon        lat                             geo  \
98      IA  OAX  6.5 -95.640000  40.750000  POINT (367315.538 -462766.274)   
248     IA  DMX  1.0 -94.050000  41.130000  POINT (498083.331 -412247.848)   
516     IA  OAX  3.0 -95.033300  40.733300  POINT (418463.471 -461731.586)   
522     IA  OAX  4.2 -96.016700  41.816700  POINT (330068.669 -345917.663)   
550     IA  DMX  3.1 -95.003100  41.418100  POINT (416553.379 -385572.801)   
...    ...  ...  ...        ...        ...                             ...   
2151    IA  DVN  0.0 -91.490770  41.653370  POINT (705700.389 -335494.907)   
2152    IA  DVN  0.0 -91.529572  41.678623  POINT (702216.226 -333031.222)   
2153    IA  DVN  0.0 -91.571810  41.651110  POINT (699033.899 -336438.358)   
2154    IA  DVN  0.0 -91.270037  42.105643   POINT (718731.11 -283500.909)   
2155    IA  DVN  0.0 -92.249930  41.262770   POINT (646941.49 -385006.898)   

      used_for_analysis     nwsli  plotme    source  xcell  yce

## 